In [2]:
using Plots, Interact, LinearAlgebra
using Random: randperm
using Statistics: mean

theme(
    :wong;
    markerstrokecolor="white", 
    markerstrokewidth=0,
    alpha=0.6,
    label=""
)

function heatmap_digit(x::AbstractMatrix; kwargs...)
    return heatmap(
        x;
        transpose=true,
        yflip=true,
        showaxis=:false,
        grid=:false,
        color=:grays,
        aspect_ratio=1.0,
        kwargs...
    )
end

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [1]:
"""
    dW, db, loss = grad_loss_1layer_1output(f_a, df_a, X, y, W, b)

Evaluate the loss function and its gradient at `X`.

**Inputs**
* `f_a`, `df_a`: activation function and its derivative, respectively
* `X`: *d* x *N* input matrix (*N* samples with dimension *d*)
* `y`: *1* x *N* output vector (function values corresponding to the *N* samples in `X`)
* `W`: *n* x *d* weight matrix, where n is the hidden layer dimension
* `b`: length-*n* bias vector

**Outputs**
* `dW`: vector of gradients with respect to weights
* `db`: gradient with respect to bias
* `loss`: loss function value
"""
function grad_loss_1layer_1output(
        f_a::Function,
        df_a::Function,
        X::AbstractMatrix,
        y::AbstractMatrix,
        W::AbstractMatrix,
        b::AbstractVector
    )

    n, d = size(W)
    N = size(X, 2)

    dW = zeros(n, d)
    db = zeros(n)
    loss = 0.0

    for k in 1:N
        error = y[k] - sum(f_a.(W * X[:, k] + b))
        for p in 1:n
            for q in 1:d
                dW[p, q] -= 2 / N * error * df_a(W[p, :]' * X[:, k] + b[p]) * X[q, k]
            end
            db[p] -= 2 / N * error * df_a(W[p, :]' * X[:, k] + b[p])
        end

        loss += 1/N * error^2
    end
    return dW, db, loss
end

grad_loss_1layer_1output

In [3]:
using Random: randperm, seed!

"""
    W, b, loss = learn2classify_sgd_1layer(f_a, df_a, grad_loss, X, y, W0, b0[,
        mu=1e-3, iters=500, batch_size=10, seed=0])

Perform stochastic gradient descent and return optimized weights `W`, bias `b`,
    and vector of loss function values `loss`.

**Inputs**
* `f_a`, `df_a`: activation function and its derivative, respectively
* `grad_loss`: gradient of the loss function
* `X`: d x N matrix representing N samples having dimension d
* `y`: 1 x N row matrix containing scalar function values for the N input samples
* `W0`: n x d initial weight matrix, where n is the hidden layer dimension
* `b0`: length-n initial bias vector
* `mu`: gradient descent step size
* `iters`: number of gradient descent iterations to perform
* `batch_size`: number of samples to use for each iteration
* `seed`: specify a random seed for batch selection

**Outputs**
* `W`: optimized matrix of weights
* `b`: optimized bias vector
* `loss`: vector of loss function values for all iterations
"""
function learn2classify_sgd_1layer(
        f_a::Function,
        df_a::Function,
        grad_loss::Function,
        X::AbstractMatrix,
        y::AbstractMatrix,
        W0::AbstractMatrix,
        b0::AbstractVector,
        mu::Number=1e-3,
        iters::Integer=500,
        batch_size::Integer=10,
        seed::Integer=0
    )
    
    (seed != 0) && seed!(seed) # use seed if provided
    
    n, d = size(W0) # number of hidden neurons, inputs
    N = size(X, 2) # number of training samples
 
    W = W0
    b = b0
    loss = zeros(iters)
    for i in 1:iters
        batch_idx = randperm(N)
        batch_idx = batch_idx[1:min(batch_size, N)]
        
        dW, db, loss_i = grad_loss(
            f_a, df_a, X[:, batch_idx], y[:, batch_idx], W, b)
        
        W -= mu * dW
        b -= mu * db

        loss[i] = loss_i
    end
    return W, b, loss
end

learn2classify_sgd_1layer

In [4]:
using Random: randperm, seed!

"""
    W, b, loss = learn2classify_asgd_1layer(f_a, df_a, grad_loss, X, y, W0, b0[,
        mu=1e-3, iters=500, batch_size=10, seed=0])

Perform accelerated stochastic gradient descent and return optimized weights
    `W`, bias `b`, and vector of loss function values `loss`.

**Inputs**
* `f_a`, `df_a`: activation function and its derivative, respectively
* `grad_loss`: gradient of the loss function
* `X`: d x N matrix representing N samples having dimension d
* `y`: 1 x N row matrix containing scalar function values for the N input samples
* `W0`: n x d initial weight matrix, where n is the hidden layer dimension
* `b0`: length-n initial bias vector
* `mu`: gradient descent step size
* `iters`: number of gradient descent iterations to perform
* `batch_size`: number of samples to use for each iteration
* `seed`: specify a random seed for batch selection

**Outputs**
* `W`: optimized matrix of weights
* `b`: optimized bias vector
* `loss`: vector of loss function values for all iterations
"""
function learn2classify_asgd_1layer(
        f_a::Function, 
        df_a::Function, 
        grad_loss::Function,
        X::AbstractMatrix, 
        y::AbstractMatrix, 
        W0::AbstractMatrix, 
        b0::AbstractVector,
        mu::Number=1e-3, 
        iters::Integer=500, 
        batch_size::Integer=10,
        seed::Integer=0
    )
    
    (seed != 0) && seed!(seed) # use seed if provided

    d = size(W0, 2) #number of inputs
    n = size(W0, 1) # number of neurons
    N = size(X, 2) # number of training samples
 
    W = W0
    b = b0
    
    loss = zeros(iters)

    lambda_k = 0
    q_k = W
    p_k = b
    for i in 1:iters
        batch_idx = randperm(N)
        batch_idx = batch_idx[1:min(batch_size, N)]
        
        dW, db, loss_i = grad_loss(f_a, df_a, X[:, batch_idx], y[:, batch_idx], W, b)
        
        q_kp1 = W - mu * dW
        p_kp1 = b - mu * db

        lambda_kp1 = (1 + sqrt(1 + 4 * lambda_k^2)) / 2
        gamma_k = (1 - lambda_k) / lambda_kp1

        W = (1 - gamma_k) * q_kp1 + gamma_k * q_k
        b = (1 - gamma_k) * p_kp1 + gamma_k * p_k

        q_k = q_kp1
        p_k = p_kp1
        lambda_k = lambda_kp1

        loss[i] = loss_i
    end
    return W, b, loss
end

learn2classify_asgd_1layer